In [1]:
import pandas as pd
import numpy as np
import torch
import pickle
from datetime import time
from tqdm.auto import tqdm
from datetime import date
from Transformer import EasyTransformer
from Transformer_config import EasyTransformerConfig
import pickle
from train import train, StockDataset, EasyTransformerTrainConfig

# from Transformer import EasyTransformer
# from Transformer import EasyTransformerConfig
# from dataclasses import dataclass
# from typing import Optional, Callable
# from torch.utils.data import Dataset, DataLoader

# Pre-process data

In [53]:
with open('../data/train_complete_9000.pkl', 'rb') as file:
    train = pickle.load(file)

In [55]:
train['time_index'] = pd.factorize(train['time'])[0]

In [17]:
# train = train[train['date'] > date(2021, 5, 1)] # Reduce size of df
# train.reset_index(inplace=True)

In [58]:
train['dt'] = pd.to_datetime(train['date'].astype(str) + ' ' + train['time'].astype(str))

In [59]:
train = train[['ticker', 'dt', 'log_rets', 'high', 'low','42_vol', 'volume', 'label', 'time_index']]

In [60]:
for l in tqdm(range(1, 43)):
    for feature in ['log_rets', '42_vol', 'volume', 'high', 'low', 'label', 'time_index']:
        train[f'L_{l}_{feature}'] = train.groupby('ticker')[feature].shift(l)

 31%|███       | 13/42 [09:51<21:59, 45.51s/it]/tmp/ipykernel_244349/1714851947.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[f'L_{l}_{feature}'] = train.groupby('ticker')[feature].shift(l)
/tmp/ipykernel_244349/1714851947.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train[f'L_{l}_{feature}'] = train.groupby('ticker')[feature].shift(l)
/tmp/ipykernel_244349/1714851947.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

In [61]:
len(train)#.isna().sum().tolist()

93297582

In [62]:
train.dropna(inplace=True)

In [63]:
len(train)

92934380

In [64]:
with open('../data/train_DL_time_index.pkl', 'wb') as file:
    pickle.dump(train, file)

# Train

In [4]:
num_features = 5
time_steps = 43

feature_cols = ['log_rets', 'high', 'low', '42_vol', 'volume']
lagged_feature_cols = [f"L_{lag}_{feat}" for lag in range(time_steps-1, 0, -1) for feat in feature_cols]
response_vars = [f'L_{l}_label' for l in range(42, 0, -1)] + ['label']

In [2]:
cfg = EasyTransformerConfig(n_layers = 3,
            d_in = 5,
            d_model = 20,
            n_ctx = 43,
            d_head = 5,
            model_name = "custom",
            n_heads = 4,
            d_mlp = 64,
            act_fn = 'relu',
            eps = 1e-5,
            use_attn_scale = True, # whether to explicitly calculate the amount each head adds to the residual stream (with a hook) and THEN add it up, vs just calculating the sum. This can be very memory intensive for large models, so defaults to False
            init_mode = "gpt2",
            normalization_type = 'LN',
            device = "cuda" if torch.cuda.is_available() else "cpu",
            attention_dir = "causal",
            seed = 42,
            initializer_range = -1.0, # This will set: self.initializer_range = 0.8 / np.sqrt(self.d_model)
            positional_embedding_type = "standard",
            d_vocab_out = 3
        ) # For binary prediction

model = EasyTransformer(cfg)

# model.load_state_dict(torch.load('../trained_models_DL/model_24000.pt', map_location=torch.device('cpu')))

Moving model to device:  cpu


In [3]:
from torchsummary import summary

# Example: Assume your model is named 'model'
summary(model, input_size=(2500000, 43, 5))  # Specify the input size (e.g., 3x224x224 for an RGB image)


RuntimeError: einsum(): the number of subscripts in the equation (3) does not match the number of dimensions (4) for operand 0 and no ellipsis was given

In [3]:
config = EasyTransformerTrainConfig(num_epochs = 20,
            batch_size = 256,
            lr = 1e-4,
            seed = 42,
            max_grad_norm = None,
            weight_decay = None, # If None we don't use AdamW
            optimizer_name = "Adam",
            device = "cuda" if torch.cuda.is_available() else "cpu",
            warmup_steps = 0,
            save_every = 2000, # After how many batches to save model (after processing save_every * batch_size samples)
            save_dir = '../trained_models_DL',
            print_every = 200, # Print loss every print_every steps (i.e., batches)
            max_steps = None
        )

In [ ]:
# params = []

# for module in model.named_parameters():
#     params.append(module)

In [35]:
with open('../data/train_DL.pkl', 'rb') as file:
        df = pickle.load(file)

dataset = StockDataset(df, lagged_feature_cols, feature_cols, response_vars)



In [ ]:
trained_model = train(
        model,
        config,
        dataset,
    )

Moving model to device:  cpu


  0%|          | 0/20 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:07,  7.02s/it]

Epoch 1 Samples 256 Step 0 Loss 1.1934888362884521



2it [00:08,  3.57s/it]
3it [00:08,  2.09s/it]
4it [00:08,  1.39s/it]
5it [00:09,  1.01s/it]
6it [00:09,  1.30it/s]
7it [00:09,  1.61it/s]
8it [00:10,  1.90it/s]
9it [00:10,  2.16it/s]
10it [00:10,  2.39it/s]
11it [00:11,  2.58it/s]
12it [00:11,  2.71it/s]
13it [00:11,  2.83it/s]
14it [00:12,  2.93it/s]
15it [00:12,  2.98it/s]
16it [00:12,  3.04it/s]
17it [00:12,  3.07it/s]
18it [00:13,  3.10it/s]
19it [00:13,  3.10it/s]
20it [00:13,  3.17it/s]
21it [00:14,  3.22it/s]
22it [00:14,  3.24it/s]
23it [00:14,  3.21it/s]
24it [00:15,  3.22it/s]
25it [00:15,  3.18it/s]
26it [00:15,  3.16it/s]
27it [00:16,  3.15it/s]
28it [00:16,  3.15it/s]
29it [00:16,  3.14it/s]
30it [00:17,  3.13it/s]
31it [00:17,  3.14it/s]
32it [00:17,  3.12it/s]
33it [00:18,  3.12it/s]
34it [00:18,  3.11it/s]
35it [00:18,  3.08it/s]
36it [00:18,  3.10it/s]
37it [00:19,  3.10it/s]
38it [00:19,  3.10it/s]
39it [00:19,  3.11it/s]
40it [00:20,  3.11it/s]
41it [00:20,  3.10it/s]
42it [00:20,  3.10it/s]
43it [00:21,  3.10it/s]

Epoch 1 Samples 51456 Step 200 Loss 1.0374337434768677


...


8802it [47:58,  3.05it/s]
8803it [47:58,  3.11it/s]
8804it [47:59,  3.14it/s]
8805it [47:59,  3.14it/s]
8806it [47:59,  3.16it/s]
8807it [48:00,  3.16it/s]
8808it [48:00,  3.16it/s]
8809it [48:00,  3.19it/s]
8810it [48:01,  3.20it/s]
8811it [48:01,  3.22it/s]
8812it [48:01,  3.21it/s]
8813it [48:01,  3.20it/s]
8814it [48:02,  3.19it/s]
8815it [48:02,  3.21it/s]
8816it [48:02,  3.21it/s]
8817it [48:03,  3.27it/s]
8818it [48:03,  3.28it/s]
8819it [48:03,  3.25it/s]
8820it [48:04,  3.24it/s]
8821it [48:04,  3.22it/s]
8822it [48:04,  3.24it/s]
8823it [48:05,  3.21it/s]
8824it [48:05,  3.21it/s]
8825it [48:05,  3.24it/s]
8826it [48:05,  3.22it/s]
8827it [48:06,  3.17it/s]
8828it [48:06,  3.16it/s]
8829it [48:06,  3.16it/s]
8830it [48:07,  3.16it/s]
8831it [48:07,  3.18it/s]
8832it [48:07,  3.18it/s]
8833it [48:09,  1.61it/s]
8834it [48:09,  1.89it/s]
8835it [48:09,  2.16it/s]
8836it [48:10,  2.39it/s]
8837it [48:10,  2.58it/s]
8838it [48:10,  2.75it/s]
8839it [48:11,  2.86it/s]
8840it [48:

Epoch 1 Samples 2304256 Step 9000 Loss 0.9631825685501099



9002it [49:03,  2.98it/s]
9003it [49:04,  3.05it/s]
9004it [49:04,  3.10it/s]
9005it [49:04,  3.13it/s]
9006it [49:05,  3.15it/s]
9007it [49:05,  3.18it/s]
9008it [49:05,  3.21it/s]
9009it [49:06,  3.19it/s]
9010it [49:06,  3.16it/s]
9011it [49:06,  3.20it/s]
9012it [49:07,  3.19it/s]
9013it [49:07,  3.17it/s]
9014it [49:07,  3.05it/s]
9015it [49:08,  3.04it/s]
9016it [49:08,  3.13it/s]
9017it [49:08,  3.15it/s]
9018it [49:09,  3.15it/s]
9019it [49:09,  3.17it/s]
9020it [49:09,  3.18it/s]
9021it [49:09,  3.19it/s]
9022it [49:10,  3.18it/s]
9023it [49:10,  3.17it/s]
9024it [49:10,  3.16it/s]
9025it [49:11,  3.16it/s]
9026it [49:11,  3.16it/s]
9027it [49:11,  3.16it/s]
9028it [49:12,  3.17it/s]
9029it [49:12,  3.21it/s]
9030it [49:12,  3.20it/s]
9031it [49:13,  3.18it/s]
9032it [49:13,  3.19it/s]
9033it [49:13,  3.18it/s]
9034it [49:14,  3.18it/s]
9035it [49:14,  3.17it/s]
9036it [49:14,  3.17it/s]
9037it [49:15,  3.16it/s]
9038it [49:15,  3.16it/s]
9039it [49:15,  3.18it/s]
9040it [49:

Epoch 1 Samples 2355456 Step 9200 Loss 0.9427117705345154



9202it [50:07,  3.01it/s]
9203it [50:07,  3.10it/s]
9204it [50:07,  3.11it/s]
9205it [50:08,  3.10it/s]
9206it [50:08,  3.10it/s]
9207it [50:08,  3.11it/s]
9208it [50:09,  3.13it/s]
9209it [50:09,  3.05it/s]
9210it [50:09,  3.07it/s]
9211it [50:09,  3.10it/s]
9212it [50:10,  3.12it/s]
9213it [50:10,  3.12it/s]
9214it [50:10,  3.13it/s]
9215it [50:11,  3.13it/s]
9216it [50:11,  3.06it/s]
  0%|          | 0/20 [50:11<?, ?it/s]
